## Get Best-Performing Hyper Parameters

In [87]:
# MONGO_COLLECTION = "CB_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "CB_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

# MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"
MONGO_COLLECTION = "SC_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

MONGO_COLLECTION

'SC_COST_INSENS_RE-RANKER_HYPER_PARAM_VD'

In [88]:
assert MONGO_COLLECTION.endswith("VD")

In [89]:
cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type"]
if "INSENS" in MONGO_COLLECTION:
    cols = [c for c in cols if c != "loss_type"]
# cols

In [90]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [91]:
df = pd.DataFrame(results)
# df.tail(3)[["asof"] + cols]

In [92]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,pa_type
8,0.806197,0.868521,0.752219,2019-06-23 16:19:24.964000,2,0.0005,300,1,0.01,1
0,0.805982,0.868699,0.751712,2019-06-23 13:43:53.839000,1,0.0005,300,1,0.01,1
1,0.805234,0.870359,0.749176,2019-06-23 16:00:51.402000,1,0.0005,300,1,0.01,2
9,0.805234,0.870359,0.749176,2019-06-23 16:21:43.095000,2,0.0005,300,1,0.01,2
30,0.805043,0.864841,0.752980,2019-06-23 18:25:36.964000,5,0.1000,300,1,0.01,1


In [93]:
# Best ones that are not 2
# df[df.best_top_n != 2][cols].head()